## Time Series
Source: Wikipedia

Time series analysis comprises methods for analyzing time series data in order to extract meaningful statistics and other characteristics of the data. Time series forecasting is the use of a model to predict future values based on previously observed values. While regression analysis is often employed in such a way as to test theories that the current values of one or more independent time series affect the current value of another time series, this type of analysis of time series is not called "time series analysis", which focuses on comparing values of a single time series or multiple dependent time series at different points in time. Interrupted time series analysis is the analysis of interventions on a single time series.

__Prediction and forecasting__  

In statistics, prediction is a part of statistical inference. One particular approach to such inference is known as predictive inference, but the prediction can be undertaken within any of the several approaches to statistical inference. Indeed, one description of statistics is that it provides a means of transferring knowledge about a sample of a population to the whole population, and to other related populations, which is not necessarily the same as prediction over time. When information is transferred across time, often to specific points in time, the process is known as forecasting.

* Fully formed statistical models for stochastic simulation purposes, so as to generate alternative versions of the time series, representing what might happen over non-specific time-periods in the future.
* Simple or fully formed statistical models to describe the likely outcome of the time series in the immediate future, given knowledge of the most recent outcomes (forecasting).
* Forecasting on time series is usually done using automated statistical software packages and programming languages, such as Apache Spark, Julia, Python, R, SAS, SPSS and many others.
* Forecasting on large scale data is done using Spark which has spark-ts as a third party package.

https://en.wikipedia.org/wiki/Time_series

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

data = [[2008, 1, 20], [2008, 2, 30], [2008, 3, 39], [2008, 4, 60],
        [2009, 1, 40], [2009, 2, 51], [2009, 3, 62], [2009, 4, 81],
        [2010, 1, 50], [2010, 2, 64], [2010, 3, 74], [2010, 4, 95]]

df = pd.DataFrame(data, columns = ['year', 'quarter', 'y'])
df